<a href="https://colab.research.google.com/github/ryanczhang7/spotifyproject/blob/master/Spotify_Test_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

**In this notebook I am querying the test data. My training data likely has bias, since all the songs are billboard 100 songs. So for the test data I will query songs from spotify featured playlists. These songs are all new songs with respect to time labels, and are all relatively popular, which is an implicit bias of any training model. So I probably have to acknowledge that my model won't perform well on less known songs.**

In [0]:
import pandas as pd

# Getting songs from 50 of Spotifies Featured Playlists

In [0]:
# Using spotipy package to interact with Spotify Web API
import spotipy

client_id = 
client_secret = 
# To access authorised Spotify data
from spotipy.oauth2 import SpotifyClientCredentials

client_credentials_manager = SpotifyClientCredentials(
    client_id=client_id, client_secret=client_secret)
# Spotify object to access API
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 

In [0]:
# Querying 50 playlists with english songs from the US that are featured 
# playlists as of April 6th, 2020 at 9AM.
playlists = sp.featured_playlists("en-US", "US", "2020-04-06T09:00:00", 50)
playlists["playlists"]["items"]

[{'collaborative': False,
  'description': "Easy feels on your way to work, or wherever you're commuting to this morning.",
  'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1DX2MyUCsl25eb'},
  'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DX2MyUCsl25eb',
  'id': '37i9dQZF1DX2MyUCsl25eb',
  'images': [{'height': None,
    'url': 'https://i.scdn.co/image/ab67706f00000002e4fc341b75061bf6e8f32019',
    'width': None}],
  'name': 'Morning Commute',
  'owner': {'display_name': 'Spotify',
   'external_urls': {'spotify': 'https://open.spotify.com/user/spotify'},
   'href': 'https://api.spotify.com/v1/users/spotify',
   'id': 'spotify',
   'type': 'user',
   'uri': 'spotify:user:spotify'},
  'primary_color': None,
  'public': None,
  'snapshot_id': 'MTU4NjEzNDMyMSwwMDAwMDAwMGQ0MWQ4Y2Q5OGYwMGIyMDRlOTgwMDk5OGVjZjg0Mjdl',
  'tracks': {'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DX2MyUCsl25eb/tracks',
   'total': 75},
  'type': 'playlist',
  'uri': 'spot

In [0]:
# Querying the tracks in first playlist and taking a look at the first item
# in the list of tracks and its items
tracks = sp.playlist_tracks(playlists["playlists"]["items"][0]["id"])["items"]

In [0]:
# I can inspect the frist track to see how to get the features I want
tracks[0]["track"]

{'album': {'album_type': 'single',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6WY7D3jk8zTrHtmkqqo5GI'},
    'href': 'https://api.spotify.com/v1/artists/6WY7D3jk8zTrHtmkqqo5GI',
    'id': '6WY7D3jk8zTrHtmkqqo5GI',
    'name': 'Maren Morris',
    'type': 'artist',
    'uri': 'spotify:artist:6WY7D3jk8zTrHtmkqqo5GI'},
   {'external_urls': {'spotify': 'https://open.spotify.com/artist/2FXC3k01G6Gw61bmprjgqS'},
    'href': 'https://api.spotify.com/v1/artists/2FXC3k01G6Gw61bmprjgqS',
    'id': '2FXC3k01G6Gw61bmprjgqS',
    'name': 'Hozier',
    'type': 'artist',
    'uri': 'spotify:artist:2FXC3k01G6Gw61bmprjgqS'}],
  'available_markets': ['AD',
   'AE',
   'AR',
   'AT',
   'AU',
   'BE',
   'BG',
   'BH',
   'BO',
   'BR',
   'CA',
   'CH',
   'CL',
   'CO',
   'CR',
   'CY',
   'CZ',
   'DE',
   'DK',
   'DO',
   'DZ',
   'EC',
   'EE',
   'EG',
   'ES',
   'FI',
   'FR',
   'GB',
   'GR',
   'GT',
   'HK',
   'HN',
   'HU',
   'ID',
   'IE',
   'IL',
   'IN

In [0]:
# This will be the template for getting the features I want
track = {}
track["id"] = a[0]["track"]["id"]
track["artist"] = a[0]["track"]["album"]["artists"][0]["name"]
track["song"] = a[0]["track"]["name"]
track["album"] = a[0]["track"]["album"]["name"]
track["date"] = a[0]["track"]["album"]["release_date"]
track["explicit"] = a[0]["track"]["explicit"]
track["popularity"] = a[0]["track"]["popularity"]
pd.json_normalize(track)

,id,artist,song,album,date,explicit,popularity
0,1yTTMcUhL7rtz08Dsgb7Qb,Maren Morris,The Bones - with Hozier,The Bones (with Hozier),2019-10-04,False,77


In [0]:
# Loop for getting all the features for all the songs in the featured playlists
songs = []
for playlist in playlists["playlists"]["items"]:
  tracks = sp.playlist_tracks(playlist["id"])["items"]
  for item in tracks:
    track = {}
    track["id"] = item["track"]["id"]
    track["artist"] = item["track"]["album"]["artists"][0]["name"]
    track["song"] = item["track"]["name"]
    track["album"] = item["track"]["album"]["name"]
    track["date"] = item["track"]["album"]["release_date"]
    track["explicit"] = item["track"]["explicit"]
    track["popularity"] = item["track"]["popularity"]

    songs.append(track)

In [0]:
test_df = pd.json_normalize(songs)
test_df

,id,artist,song,album,date,explicit,popularity
0,1yTTMcUhL7rtz08Dsgb7Qb,Maren Morris,The Bones - with Hozier,The Bones (with Hozier),2019-10-04,False,77
1,5CalS8Gn69OOrR9aiw0ZO9,The Head and the Heart,Honeybee,Living Mirage,2019-05-17,False,71
2,6UnCGAEmrbGIOSmGRZQ1M2,Maggie Rogers,Light On,Heard It In A Past Life,2019-01-18,False,74
3,3V0nnQhqvbE3JmiDdnzQFQ,Noah Cyrus,July (feat. Leon Bridges),July (feat. Leon Bridges),2019-11-08,False,73
4,4MIPNDbxVPWdLwH6A4nsiY,Valley,There's Still A Light In The House,MAYBE,2019-09-17,False,62
...,...,...,...,...,...,...,...
895,5eek2X5459T1HoYJk2CKXv,Papa Roach,Last Resort,Infest,2000-04-25,True,17
896,24tO365YW6lcZr1hN4Ukzj,Silverchair,Tomorrow,Frogstomp (Deluxe Edition) [Remastered],1995-03-27,False,61
897,6wpHmxmxPa3N3Gi3aGdTJL,Atreyu,The Time Is Now,The Time Is Now,2018-09-07,False,62
898,3bwAS8cN7Xtzvv5UurJOOX,Audioslave,I Am the Highway,Audioslave,2002-11-17,False,66


In [0]:
test_df.isnull().sum()

id            0
artist        0
song          0
album         0
date          0
explicit      0
popularity    0
dtype: int64

In [0]:
import re
# Formatting the names of songs to match the training set. 
# I formatted the names of the songs that way in the training set
# so I can easily web scrape using a metrolyrics url generator.

test_df["song"] = test_df["song"].str.replace(r' \([^)]*\)', "")
test_df["song"] = test_df["song"].str.replace(r' -( .+)+$', "")
test_df

,id,artist,song,album,date,explicit,popularity
0,1yTTMcUhL7rtz08Dsgb7Qb,Maren Morris,The Bones,The Bones (with Hozier),2019-10-04,False,77
1,5CalS8Gn69OOrR9aiw0ZO9,The Head and the Heart,Honeybee,Living Mirage,2019-05-17,False,71
2,6UnCGAEmrbGIOSmGRZQ1M2,Maggie Rogers,Light On,Heard It In A Past Life,2019-01-18,False,74
3,3V0nnQhqvbE3JmiDdnzQFQ,Noah Cyrus,July,July (feat. Leon Bridges),2019-11-08,False,73
4,4MIPNDbxVPWdLwH6A4nsiY,Valley,There's Still A Light In The House,MAYBE,2019-09-17,False,62
...,...,...,...,...,...,...,...
895,5eek2X5459T1HoYJk2CKXv,Papa Roach,Last Resort,Infest,2000-04-25,True,17
896,24tO365YW6lcZr1hN4Ukzj,Silverchair,Tomorrow,Frogstomp (Deluxe Edition) [Remastered],1995-03-27,False,61
897,6wpHmxmxPa3N3Gi3aGdTJL,Atreyu,The Time Is Now,The Time Is Now,2018-09-07,False,62
898,3bwAS8cN7Xtzvv5UurJOOX,Audioslave,I Am the Highway,Audioslave,2002-11-17,False,66


**It is possible for some songs to appear twice.**

In [0]:
test_df['id'].value_counts()

2amPRWMdtrMrGG1vafKvxl    2
4kbj5MwxO1bq9wjT5g9HaA    1
1AEYT6VxrxXPMoQUxsY0E4    1
0UHiH4dy1hXULYl8CgNxRV    1
0s5DNDV6LAKn2u2Ufa3VjQ    1
                         ..
3hqCeRRTNreQ9CFtpk19JR    1
322tcqPhma7F6hfCeLisx0    1
6p8R2eD9v5PeoNqSLVD4JX    1
7Gr61S9VTm8fcYfAo4Fd9g    1
4e9hUiLsN4mx61ARosFi7p    1
Name: id, Length: 899, dtype: int64

In [0]:
test_df[test_df['id']=="2amPRWMdtrMrGG1vafKvxl"]

,id,artist,song,album,date,explicit,popularity
24,2amPRWMdtrMrGG1vafKvxl,Leon Bridges,Beyond,Spotify Singles,2018-07-25,False,57
280,2amPRWMdtrMrGG1vafKvxl,Leon Bridges,Beyond,Spotify Singles,2018-07-25,False,57


In [0]:
# New df without the duplicated song
test_df = test_df.drop_duplicates().reset_index().iloc[:,1:]
test_df

,id,artist,song,album,date,explicit,popularity
0,1yTTMcUhL7rtz08Dsgb7Qb,Maren Morris,The Bones,The Bones (with Hozier),2019-10-04,False,77
1,5CalS8Gn69OOrR9aiw0ZO9,The Head and the Heart,Honeybee,Living Mirage,2019-05-17,False,71
2,6UnCGAEmrbGIOSmGRZQ1M2,Maggie Rogers,Light On,Heard It In A Past Life,2019-01-18,False,74
3,3V0nnQhqvbE3JmiDdnzQFQ,Noah Cyrus,July,July (feat. Leon Bridges),2019-11-08,False,73
4,4MIPNDbxVPWdLwH6A4nsiY,Valley,There's Still A Light In The House,MAYBE,2019-09-17,False,62
...,...,...,...,...,...,...,...
894,5eek2X5459T1HoYJk2CKXv,Papa Roach,Last Resort,Infest,2000-04-25,True,17
895,24tO365YW6lcZr1hN4Ukzj,Silverchair,Tomorrow,Frogstomp (Deluxe Edition) [Remastered],1995-03-27,False,61
896,6wpHmxmxPa3N3Gi3aGdTJL,Atreyu,The Time Is Now,The Time Is Now,2018-09-07,False,62
897,3bwAS8cN7Xtzvv5UurJOOX,Audioslave,I Am the Highway,Audioslave,2002-11-17,False,66


In [0]:
# This is the audio features function I wrote for the original 1050 songs

def audio_features(df):
  tracks = []

  for id in df["id"]:
    track_features = {}
    # pull audio features per track
    features = sp.audio_features(id)
    
    # assign dictionary values for each audio feature
    track_features["id"] = id
    track_features['key'] = features[0]['key']
    track_features['mode'] = features[0]['mode']
    track_features['time_signature'] = features[0]['time_signature']
    track_features['acousticness'] = features[0]['acousticness']
    track_features['danceability'] = features[0]['danceability']
    track_features['energy'] = features[0]['energy']
    track_features['instrumentalness'] = features[0]['instrumentalness']
    track_features['liveness'] = features[0]['liveness']
    track_features['loudness'] = features[0]['loudness']
    track_features['speechiness'] = features[0]['speechiness']
    track_features['tempo'] = features[0]['tempo']
    track_features['valence'] = features[0]['valence']
    
    tracks.append(track_features)

  return tracks

In [0]:
audio = audio_features(test_df)
df_audio = pd.json_normalize(audio)

retrying after...3secs
retrying after...5secs
retrying after...2secs
retrying after...3secs
retrying after...5secs
retrying after...1secs


In [0]:
df_audio

,id,key,mode,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
0,1yTTMcUhL7rtz08Dsgb7Qb,11,0,4,0.286000,0.561,0.597,0.000000,0.0979,-6.000,0.0405,76.826,0.355
1,5CalS8Gn69OOrR9aiw0ZO9,9,1,4,0.118000,0.642,0.520,0.000512,0.5520,-7.886,0.0387,136.014,0.488
2,6UnCGAEmrbGIOSmGRZQ1M2,2,1,4,0.201000,0.657,0.569,0.000014,0.1260,-6.287,0.0542,102.054,0.399
3,3V0nnQhqvbE3JmiDdnzQFQ,9,0,4,0.853000,0.708,0.211,0.000000,0.1310,-9.603,0.0424,72.500,0.362
4,4MIPNDbxVPWdLwH6A4nsiY,4,1,4,0.024000,0.620,0.655,0.000042,0.3010,-8.525,0.0455,109.987,0.335
...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,5eek2X5459T1HoYJk2CKXv,4,0,4,0.000504,0.581,0.887,0.001110,0.2680,-3.659,0.0624,90.578,0.724
895,24tO365YW6lcZr1hN4Ukzj,2,1,3,0.000658,0.292,0.714,0.000109,0.1020,-4.919,0.0316,151.412,0.462
896,6wpHmxmxPa3N3Gi3aGdTJL,8,1,4,0.009260,0.481,0.944,0.000328,0.2780,-3.665,0.2310,165.020,0.396
897,3bwAS8cN7Xtzvv5UurJOOX,9,0,4,0.105000,0.341,0.662,0.000000,0.1130,-5.023,0.0335,174.875,0.200


In [0]:
# Now to merge the dataframes together.
# I kept id in the audio dataframe for clean merging
test_df = test_df.merge(df_audio, on="id", how="outer")
test_df

,id,artist,song,album,date,explicit,popularity,key,mode,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
0,1yTTMcUhL7rtz08Dsgb7Qb,Maren Morris,The Bones,The Bones (with Hozier),2019-10-04,False,77,11,0,4,0.286000,0.561,0.597,0.000000,0.0979,-6.000,0.0405,76.826,0.355
1,5CalS8Gn69OOrR9aiw0ZO9,The Head and the Heart,Honeybee,Living Mirage,2019-05-17,False,71,9,1,4,0.118000,0.642,0.520,0.000512,0.5520,-7.886,0.0387,136.014,0.488
2,6UnCGAEmrbGIOSmGRZQ1M2,Maggie Rogers,Light On,Heard It In A Past Life,2019-01-18,False,74,2,1,4,0.201000,0.657,0.569,0.000014,0.1260,-6.287,0.0542,102.054,0.399
3,3V0nnQhqvbE3JmiDdnzQFQ,Noah Cyrus,July,July (feat. Leon Bridges),2019-11-08,False,73,9,0,4,0.853000,0.708,0.211,0.000000,0.1310,-9.603,0.0424,72.500,0.362
4,4MIPNDbxVPWdLwH6A4nsiY,Valley,There's Still A Light In The House,MAYBE,2019-09-17,False,62,4,1,4,0.024000,0.620,0.655,0.000042,0.3010,-8.525,0.0455,109.987,0.335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,5eek2X5459T1HoYJk2CKXv,Papa Roach,Last Resort,Infest,2000-04-25,True,17,4,0,4,0.000504,0.581,0.887,0.001110,0.2680,-3.659,0.0624,90.578,0.724
895,24tO365YW6lcZr1hN4Ukzj,Silverchair,Tomorrow,Frogstomp (Deluxe Edition) [Remastered],1995-03-27,False,61,2,1,3,0.000658,0.292,0.714,0.000109,0.1020,-4.919,0.0316,151.412,0.462
896,6wpHmxmxPa3N3Gi3aGdTJL,Atreyu,The Time Is Now,The Time Is Now,2018-09-07,False,62,8,1,4,0.009260,0.481,0.944,0.000328,0.2780,-3.665,0.2310,165.020,0.396
897,3bwAS8cN7Xtzvv5UurJOOX,Audioslave,I Am the Highway,Audioslave,2002-11-17,False,66,9,0,4,0.105000,0.341,0.662,0.000000,0.1130,-5.023,0.0335,174.875,0.200


In [0]:
test_df.to_csv("/content/drive/My Drive/test_df_spotify_incomplete.csv", 
               index=False)

# Web Scraping Lyrics from Metrolyrics

In [0]:
test_df = pd.read_csv("/content/drive/My Drive/test_df_spotify_incomplete.csv")
test_df

,id,artist,song,album,date,explicit,popularity,key,mode,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
0,1yTTMcUhL7rtz08Dsgb7Qb,Maren Morris,The Bones,The Bones (with Hozier),2019-10-04,False,77,11,0,4,0.286000,0.561,0.597,0.000000,0.0979,-6.000,0.0405,76.826,0.355
1,5CalS8Gn69OOrR9aiw0ZO9,The Head and the Heart,Honeybee,Living Mirage,2019-05-17,False,71,9,1,4,0.118000,0.642,0.520,0.000512,0.5520,-7.886,0.0387,136.014,0.488
2,6UnCGAEmrbGIOSmGRZQ1M2,Maggie Rogers,Light On,Heard It In A Past Life,2019-01-18,False,74,2,1,4,0.201000,0.657,0.569,0.000014,0.1260,-6.287,0.0542,102.054,0.399
3,3V0nnQhqvbE3JmiDdnzQFQ,Noah Cyrus,July,July (feat. Leon Bridges),2019-11-08,False,73,9,0,4,0.853000,0.708,0.211,0.000000,0.1310,-9.603,0.0424,72.500,0.362
4,4MIPNDbxVPWdLwH6A4nsiY,Valley,There's Still A Light In The House,MAYBE,2019-09-17,False,62,4,1,4,0.024000,0.620,0.655,0.000042,0.3010,-8.525,0.0455,109.987,0.335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,5eek2X5459T1HoYJk2CKXv,Papa Roach,Last Resort,Infest,2000-04-25,True,17,4,0,4,0.000504,0.581,0.887,0.001110,0.2680,-3.659,0.0624,90.578,0.724
895,24tO365YW6lcZr1hN4Ukzj,Silverchair,Tomorrow,Frogstomp (Deluxe Edition) [Remastered],1995-03-27,False,61,2,1,3,0.000658,0.292,0.714,0.000109,0.1020,-4.919,0.0316,151.412,0.462
896,6wpHmxmxPa3N3Gi3aGdTJL,Atreyu,The Time Is Now,The Time Is Now,2018-09-07,False,62,8,1,4,0.009260,0.481,0.944,0.000328,0.2780,-3.665,0.2310,165.020,0.396
897,3bwAS8cN7Xtzvv5UurJOOX,Audioslave,I Am the Highway,Audioslave,2002-11-17,False,66,9,0,4,0.105000,0.341,0.662,0.000000,0.1130,-5.023,0.0335,174.875,0.200


In [0]:
# Web scraping the lyrics for songs off of metrolyrics.com
# This is what I used for the original 1050 songs, and for the training data
import time
import string
from bs4 import BeautifulSoup
import requests

songs = zip(test_df["artist"], test_df["song"])

song_lyrics = []
for song in songs:
  url = "https://www.metrolyrics.com/"
  
  # Dictionary for the song lyrics
  lyrics_dict = {}

  try:
    a = list(song)
    b = a[1].split() + ["lyrics"] + a[0].split()
    c = [c.translate(str.maketrans('', '', string.punctuation)) for c in b]
    d = "-".join(c).lower()
    d += ".html"
    url += d

    content = requests.get(url)
    soup = BeautifulSoup(content.content, 'html.parser')

    # Metrolyrics has nice label "verse" for div, which makes scraping easy
    a = soup.findAll("p", class_="verse")
    # Song lyrics are text separated by html tags, so I join them with a space
    c = " ".join([b.text for b in a])
    # Removing verse labels, and adding spaces to \n for even spacing
    d = re.sub(r'\[.*?\]', "", c).strip().replace("\n", " \n ").lower()
    # Removing punctuation
    e = d.translate(str.maketrans('', '', string.punctuation))
    # Removing parentheses around any song lyrics
    f = re.sub('[()]', '', e)
    # Replacing multiple spaces with just one, this will be chain_lyrics
    g = re.sub(r' +', " ", f)
    # This will be the clean lyrics without \n
    h = re.sub(r' +', " ", g.replace("\n", " "))
    lyrics_dict["chain_lyrics"] = g
    lyrics_dict["lyrics"] = h

    song_lyrics.append(lyrics_dict)

  # Try except, in case there is an problem with the scraping append empty dict
  except Exception as e:
    song_lyrics.append({})
  
  time.sleep(0.1)

lyrics_df = pd.json_normalize(song_lyrics)

In [0]:
# So I only managed to get 510 lyrics
lyrics_df[lyrics_df["lyrics"]!=""]

,chain_lyrics,lyrics
0,were in the homestretch of the high times \n w...,were in the homestretch of the high times we t...
2,would you believe me now \n if i told you i go...,would you believe me now if i told you i got c...
3,ive been holding my breath \n ive been countin...,ive been holding my breath ive been counting t...
11,born in a hurry always late \n havent been ear...,born in a hurry always late havent been early ...
13,i guess i just feel like \n i guess i just fee...,i guess i just feel like i guess i just feel l...
...,...,...
891,keep you in the dark \n you know they all pret...,keep you in the dark you know they all pretend...
892,turn off your television \n theyre selling sup...,turn off your television theyre selling supers...
894,cut my life into pieces \n this is my last res...,cut my life into pieces this is my last resort...
895,its twelve oclock and its a wonderful day \n i...,its twelve oclock and its a wonderful day i kn...


In [0]:
test_df = test_df.merge(lyrics_df, left_index=True, right_index=True,
                        how="outer")
test_df

,id,artist,song,album,date,explicit,popularity,key,mode,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,chain_lyrics,lyrics
0,1yTTMcUhL7rtz08Dsgb7Qb,Maren Morris,The Bones,The Bones (with Hozier),2019-10-04,False,77,11,0,4,0.286000,0.561,0.597,0.000000,0.0979,-6.000,0.0405,76.826,0.355,were in the homestretch of the high times \n w...,were in the homestretch of the high times we t...
1,5CalS8Gn69OOrR9aiw0ZO9,The Head and the Heart,Honeybee,Living Mirage,2019-05-17,False,71,9,1,4,0.118000,0.642,0.520,0.000512,0.5520,-7.886,0.0387,136.014,0.488,,
2,6UnCGAEmrbGIOSmGRZQ1M2,Maggie Rogers,Light On,Heard It In A Past Life,2019-01-18,False,74,2,1,4,0.201000,0.657,0.569,0.000014,0.1260,-6.287,0.0542,102.054,0.399,would you believe me now \n if i told you i go...,would you believe me now if i told you i got c...
3,3V0nnQhqvbE3JmiDdnzQFQ,Noah Cyrus,July,July (feat. Leon Bridges),2019-11-08,False,73,9,0,4,0.853000,0.708,0.211,0.000000,0.1310,-9.603,0.0424,72.500,0.362,ive been holding my breath \n ive been countin...,ive been holding my breath ive been counting t...
4,4MIPNDbxVPWdLwH6A4nsiY,Valley,There's Still A Light In The House,MAYBE,2019-09-17,False,62,4,1,4,0.024000,0.620,0.655,0.000042,0.3010,-8.525,0.0455,109.987,0.335,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,5eek2X5459T1HoYJk2CKXv,Papa Roach,Last Resort,Infest,2000-04-25,True,17,4,0,4,0.000504,0.581,0.887,0.001110,0.2680,-3.659,0.0624,90.578,0.724,cut my life into pieces \n this is my last res...,cut my life into pieces this is my last resort...
895,24tO365YW6lcZr1hN4Ukzj,Silverchair,Tomorrow,Frogstomp (Deluxe Edition) [Remastered],1995-03-27,False,61,2,1,3,0.000658,0.292,0.714,0.000109,0.1020,-4.919,0.0316,151.412,0.462,its twelve oclock and its a wonderful day \n i...,its twelve oclock and its a wonderful day i kn...
896,6wpHmxmxPa3N3Gi3aGdTJL,Atreyu,The Time Is Now,The Time Is Now,2018-09-07,False,62,8,1,4,0.009260,0.481,0.944,0.000328,0.2780,-3.665,0.2310,165.020,0.396,,
897,3bwAS8cN7Xtzvv5UurJOOX,Audioslave,I Am the Highway,Audioslave,2002-11-17,False,66,9,0,4,0.105000,0.341,0.662,0.000000,0.1130,-5.023,0.0335,174.875,0.200,,


In [0]:
# This is the progress so far
test_df = test_df[test_df["lyrics"]!=""]
test_df = test_df.reset_index().iloc[:,1:]

In [0]:
test_df.to_csv("/content/drive/My Drive/test_df_spotify_incomplete.csv", 
               index=False)

# Sentiment Analysis from Google Natural Language API

In [0]:
test_df = pd.read_csv("/content/drive/My Drive/test_df_spotify_incomplete.csv")
test_df

,id,artist,song,album,date,explicit,popularity,key,mode,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,chain_lyrics,lyrics
0,1yTTMcUhL7rtz08Dsgb7Qb,Maren Morris,The Bones,The Bones (with Hozier),2019-10-04,False,77,11,0,4,0.286000,0.561,0.597,0.000000,0.0979,-6.000,0.0405,76.826,0.355,were in the homestretch of the high times \n w...,were in the homestretch of the high times we t...
1,6UnCGAEmrbGIOSmGRZQ1M2,Maggie Rogers,Light On,Heard It In A Past Life,2019-01-18,False,74,2,1,4,0.201000,0.657,0.569,0.000014,0.1260,-6.287,0.0542,102.054,0.399,would you believe me now \n if i told you i go...,would you believe me now if i told you i got c...
2,3V0nnQhqvbE3JmiDdnzQFQ,Noah Cyrus,July,July (feat. Leon Bridges),2019-11-08,False,73,9,0,4,0.853000,0.708,0.211,0.000000,0.1310,-9.603,0.0424,72.500,0.362,ive been holding my breath \n ive been countin...,ive been holding my breath ive been counting t...
3,6ET9kf9riLETWs9lePUEAI,Kacey Musgraves,Slow Burn,Golden Hour,2018-03-30,False,69,2,1,4,0.462000,0.555,0.430,0.000191,0.0869,-9.105,0.0295,147.613,0.274,born in a hurry always late \n havent been ear...,born in a hurry always late havent been early ...
4,1HNywa8Bk0TmzRwQrzhtPo,John Mayer,I Guess I Just Feel Like,I Guess I Just Feel Like,2019-02-22,False,73,4,1,4,0.615000,0.556,0.528,0.096400,0.1220,-8.033,0.0266,159.821,0.163,i guess i just feel like \n i guess i just fee...,i guess i just feel like i guess i just feel l...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,7x8dCjCr0x6x2lXKujYD34,Foo Fighters,The Pretender,"Echoes, Silence, Patience & Grace",2007-09-25,False,75,9,1,4,0.000917,0.433,0.959,0.000000,0.0280,-4.040,0.0431,172.984,0.365,keep you in the dark \n you know they all pret...,keep you in the dark you know they all pretend...
506,3eDJXbkSGulxTuB2L7fP9B,Crobot,Low Life,Motherbrain,2019-08-23,False,59,6,1,4,0.000800,0.546,0.911,0.000077,0.1310,-4.801,0.0620,76.008,0.757,turn off your television \n theyre selling sup...,turn off your television theyre selling supers...
507,5eek2X5459T1HoYJk2CKXv,Papa Roach,Last Resort,Infest,2000-04-25,True,17,4,0,4,0.000504,0.581,0.887,0.001110,0.2680,-3.659,0.0624,90.578,0.724,cut my life into pieces \n this is my last res...,cut my life into pieces this is my last resort...
508,24tO365YW6lcZr1hN4Ukzj,Silverchair,Tomorrow,Frogstomp (Deluxe Edition) [Remastered],1995-03-27,False,61,2,1,3,0.000658,0.292,0.714,0.000109,0.1020,-4.919,0.0316,151.412,0.462,its twelve oclock and its a wonderful day \n i...,its twelve oclock and its a wonderful day i kn...


In [0]:
# Grabbing sentiment analysis or lyrics from Google Cloud Natural Language API.
# I think Google likely will perform better than perhaps a package
from googleapiclient.discovery import build
lservice = build('language', 'v1beta1', developerKey=APIKEY)

sentiments = []
for lyric in test_df["lyrics"]:
  sentiment = {}
 
  try:
    response = lservice.documents().analyzeSentiment(
      body={
        'document': {
          'type': 'PLAIN_TEXT',
          'content': lyric
        }
      }).execute()
    sentiment["polarity"] = response['documentSentiment']['polarity']
    sentiment["magnitude"] = response['documentSentiment']['magnitude']
    sentiments.append(sentiment)

  except Exception as e:
    sentiments.append({})

In [0]:
sentiment_df = pd.json_normalize(sentiments)
sentiment_df

,polarity,magnitude
0,-1.0,1.0
1,-1.0,0.1
2,1.0,0.1
3,-1.0,0.2
4,1.0,0.4
...,...,...
505,-1.0,0.9
506,-1.0,1.1
507,-1.0,1.6
508,-1.0,0.5


In [0]:
test_df = test_df.merge(sentiment_df, left_index=True, right_index=True,
                        how="outer")
test_df

,id,artist,song,album,date,explicit,popularity,key,mode,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,chain_lyrics,lyrics,polarity,magnitude
0,1yTTMcUhL7rtz08Dsgb7Qb,Maren Morris,The Bones,The Bones (with Hozier),2019-10-04,False,77,11,0,4,0.286000,0.561,0.597,0.000000,0.0979,-6.000,0.0405,76.826,0.355,were in the homestretch of the high times \n w...,were in the homestretch of the high times we t...,-1.0,1.0
1,6UnCGAEmrbGIOSmGRZQ1M2,Maggie Rogers,Light On,Heard It In A Past Life,2019-01-18,False,74,2,1,4,0.201000,0.657,0.569,0.000014,0.1260,-6.287,0.0542,102.054,0.399,would you believe me now \n if i told you i go...,would you believe me now if i told you i got c...,-1.0,0.1
2,3V0nnQhqvbE3JmiDdnzQFQ,Noah Cyrus,July,July (feat. Leon Bridges),2019-11-08,False,73,9,0,4,0.853000,0.708,0.211,0.000000,0.1310,-9.603,0.0424,72.500,0.362,ive been holding my breath \n ive been countin...,ive been holding my breath ive been counting t...,1.0,0.1
3,6ET9kf9riLETWs9lePUEAI,Kacey Musgraves,Slow Burn,Golden Hour,2018-03-30,False,69,2,1,4,0.462000,0.555,0.430,0.000191,0.0869,-9.105,0.0295,147.613,0.274,born in a hurry always late \n havent been ear...,born in a hurry always late havent been early ...,-1.0,0.2
4,1HNywa8Bk0TmzRwQrzhtPo,John Mayer,I Guess I Just Feel Like,I Guess I Just Feel Like,2019-02-22,False,73,4,1,4,0.615000,0.556,0.528,0.096400,0.1220,-8.033,0.0266,159.821,0.163,i guess i just feel like \n i guess i just fee...,i guess i just feel like i guess i just feel l...,1.0,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,7x8dCjCr0x6x2lXKujYD34,Foo Fighters,The Pretender,"Echoes, Silence, Patience & Grace",2007-09-25,False,75,9,1,4,0.000917,0.433,0.959,0.000000,0.0280,-4.040,0.0431,172.984,0.365,keep you in the dark \n you know they all pret...,keep you in the dark you know they all pretend...,-1.0,0.9
506,3eDJXbkSGulxTuB2L7fP9B,Crobot,Low Life,Motherbrain,2019-08-23,False,59,6,1,4,0.000800,0.546,0.911,0.000077,0.1310,-4.801,0.0620,76.008,0.757,turn off your television \n theyre selling sup...,turn off your television theyre selling supers...,-1.0,1.1
507,5eek2X5459T1HoYJk2CKXv,Papa Roach,Last Resort,Infest,2000-04-25,True,17,4,0,4,0.000504,0.581,0.887,0.001110,0.2680,-3.659,0.0624,90.578,0.724,cut my life into pieces \n this is my last res...,cut my life into pieces this is my last resort...,-1.0,1.6
508,24tO365YW6lcZr1hN4Ukzj,Silverchair,Tomorrow,Frogstomp (Deluxe Edition) [Remastered],1995-03-27,False,61,2,1,3,0.000658,0.292,0.714,0.000109,0.1020,-4.919,0.0316,151.412,0.462,its twelve oclock and its a wonderful day \n i...,its twelve oclock and its a wonderful day i kn...,-1.0,0.5


In [0]:
# I lost only one song because it was in a language not supported by the API
test_df = test_df[test_df["polarity"].notnull()]
test_df = test_df.reset_index().iloc[:,1:]

In [0]:
test_df

,id,artist,song,album,date,explicit,popularity,key,mode,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,chain_lyrics,lyrics,polarity,magnitude
0,1yTTMcUhL7rtz08Dsgb7Qb,Maren Morris,The Bones,The Bones (with Hozier),2019-10-04,False,77,11,0,4,0.286000,0.561,0.597,0.000000,0.0979,-6.000,0.0405,76.826,0.355,were in the homestretch of the high times \n w...,were in the homestretch of the high times we t...,-1.0,1.0
1,6UnCGAEmrbGIOSmGRZQ1M2,Maggie Rogers,Light On,Heard It In A Past Life,2019-01-18,False,74,2,1,4,0.201000,0.657,0.569,0.000014,0.1260,-6.287,0.0542,102.054,0.399,would you believe me now \n if i told you i go...,would you believe me now if i told you i got c...,-1.0,0.1
2,3V0nnQhqvbE3JmiDdnzQFQ,Noah Cyrus,July,July (feat. Leon Bridges),2019-11-08,False,73,9,0,4,0.853000,0.708,0.211,0.000000,0.1310,-9.603,0.0424,72.500,0.362,ive been holding my breath \n ive been countin...,ive been holding my breath ive been counting t...,1.0,0.1
3,6ET9kf9riLETWs9lePUEAI,Kacey Musgraves,Slow Burn,Golden Hour,2018-03-30,False,69,2,1,4,0.462000,0.555,0.430,0.000191,0.0869,-9.105,0.0295,147.613,0.274,born in a hurry always late \n havent been ear...,born in a hurry always late havent been early ...,-1.0,0.2
4,1HNywa8Bk0TmzRwQrzhtPo,John Mayer,I Guess I Just Feel Like,I Guess I Just Feel Like,2019-02-22,False,73,4,1,4,0.615000,0.556,0.528,0.096400,0.1220,-8.033,0.0266,159.821,0.163,i guess i just feel like \n i guess i just fee...,i guess i just feel like i guess i just feel l...,1.0,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,7x8dCjCr0x6x2lXKujYD34,Foo Fighters,The Pretender,"Echoes, Silence, Patience & Grace",2007-09-25,False,75,9,1,4,0.000917,0.433,0.959,0.000000,0.0280,-4.040,0.0431,172.984,0.365,keep you in the dark \n you know they all pret...,keep you in the dark you know they all pretend...,-1.0,0.9
505,3eDJXbkSGulxTuB2L7fP9B,Crobot,Low Life,Motherbrain,2019-08-23,False,59,6,1,4,0.000800,0.546,0.911,0.000077,0.1310,-4.801,0.0620,76.008,0.757,turn off your television \n theyre selling sup...,turn off your television theyre selling supers...,-1.0,1.1
506,5eek2X5459T1HoYJk2CKXv,Papa Roach,Last Resort,Infest,2000-04-25,True,17,4,0,4,0.000504,0.581,0.887,0.001110,0.2680,-3.659,0.0624,90.578,0.724,cut my life into pieces \n this is my last res...,cut my life into pieces this is my last resort...,-1.0,1.6
507,24tO365YW6lcZr1hN4Ukzj,Silverchair,Tomorrow,Frogstomp (Deluxe Edition) [Remastered],1995-03-27,False,61,2,1,3,0.000658,0.292,0.714,0.000109,0.1020,-4.919,0.0316,151.412,0.462,its twelve oclock and its a wonderful day \n i...,its twelve oclock and its a wonderful day i kn...,-1.0,0.5


In [0]:
test_df.to_csv("/content/drive/My Drive/test_df_spotify_incomplete.csv", 
               index=False)

# Personality Insights with IBM Watson Cloud API

In [0]:
pip install --upgrade "ibm-watson>=4.3.0"

     |████████████████████████████████| 368kB 2.8MB/s 
     |████████████████████████████████| 204kB 47.0MB/s 
  Created wheel for ibm-watson: filename=ibm_watson-4.3.0-cp36-none-any.whl size=354170 sha256=89985c4d5c53970fe856a2e52a0f9db0e2310035c6dd3a43386052e61ad00a44
  Stored in directory: /root/.cache/pip/wheels/b7/10/b4/509307276e9d558a6b06d8e69f8faf6cfbdf2d1b83abb6b8d4
  Created wheel for ibm-cloud-sdk-core: filename=ibm_cloud_sdk_core-1.5.1-cp36-none-any.whl size=44491 sha256=0f20e117ec2795a20ab69160f4560573fd7f0243824de1703fe928a5fd3e9d2c
  Stored in directory: /root/.cache/pip/wheels/6a/42/50/f96888116b329578304f9dda4693cef6f3e76e18272d22cb6c
Successfully built ibm-watson ibm-cloud-sdk-core


In [0]:
# Practicing getting one result for one text "document"
from ibm_watson import PersonalityInsightsV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
# Authenticator object
authenticator = IAMAuthenticator(APIKEY)
personality_insights = PersonalityInsightsV3(
    version= '2017-10-13',
    authenticator=authenticator
)
# Dallas service url, closest to SLO
personality_insights.set_service_url(
    'https://api.us-south.personality-insights.watson.cloud.ibm.com')

profile = personality_insights.profile(
        test_df.loc[0]["lyrics"],
        accept='application/json',
        content_type='text/html',
).get_result()

**The response json contains 5 categories: Agreeableness, Conscientiousness, Extraversion, Emotional range, and Openness. Each category has a list of personality types for which a percentile is calcualted. I will look at the result for one text document and see which personality types I want to extract (whatever ones I think are good predictors. For the original 1050 dataset, I only selected 10 features, but now I want more.**

In [0]:
# Inspecting the results for each of the big 5 categories
profile["personality"][0]["children"]

[{'category': 'personality',
  'name': 'Adventurousness',
  'percentile': 0.46131002724087716,
  'significant': True,
  'trait_id': 'facet_adventurousness'},
 {'category': 'personality',
  'name': 'Artistic interests',
  'percentile': 0.9877741037522365,
  'significant': True,
  'trait_id': 'facet_artistic_interests'},
 {'category': 'personality',
  'name': 'Emotionality',
  'percentile': 0.607614001899865,
  'significant': True,
  'trait_id': 'facet_emotionality'},
 {'category': 'personality',
  'name': 'Imagination',
  'percentile': 0.8574144920593829,
  'significant': True,
  'trait_id': 'facet_imagination'},
 {'category': 'personality',
  'name': 'Intellect',
  'percentile': 0.9743604342330916,
  'significant': True,
  'trait_id': 'facet_intellect'},
 {'category': 'personality',
  'name': 'Authority-challenging',
  'percentile': 0.8182154004283968,
  'significant': True,
  'trait_id': 'facet_liberalism'}]

**I want artistic, emotionality, and imagination, authority-challenging which are at
profile["personality"][0]["children"] items 1,2,3,5.**

In [0]:
profile["personality"][1]["children"]

[{'category': 'personality',
  'name': 'Achievement striving',
  'percentile': 0.6729313039328733,
  'significant': True,
  'trait_id': 'facet_achievement_striving'},
 {'category': 'personality',
  'name': 'Cautiousness',
  'percentile': 0.9666315253733663,
  'significant': True,
  'trait_id': 'facet_cautiousness'},
 {'category': 'personality',
  'name': 'Dutifulness',
  'percentile': 0.7785526837178879,
  'significant': True,
  'trait_id': 'facet_dutifulness'},
 {'category': 'personality',
  'name': 'Orderliness',
  'percentile': 0.9939325367844118,
  'significant': True,
  'trait_id': 'facet_orderliness'},
 {'category': 'personality',
  'name': 'Self-discipline',
  'percentile': 0.7220046066807455,
  'significant': True,
  'trait_id': 'facet_self_discipline'},
 {'category': 'personality',
  'name': 'Self-efficacy',
  'percentile': 0.7222490485236819,
  'significant': True,
  'trait_id': 'facet_self_efficacy'}]

**I don't necessarily need anything here.**

In [0]:
profile["personality"][2]["children"]

[{'category': 'personality',
  'name': 'Activity level',
  'percentile': 0.16770487586608673,
  'significant': True,
  'trait_id': 'facet_activity_level'},
 {'category': 'personality',
  'name': 'Assertiveness',
  'percentile': 0.48057541931441816,
  'significant': True,
  'trait_id': 'facet_assertiveness'},
 {'category': 'personality',
  'name': 'Cheerfulness',
  'percentile': 0.6921443916285832,
  'significant': True,
  'trait_id': 'facet_cheerfulness'},
 {'category': 'personality',
  'name': 'Excitement-seeking',
  'percentile': 0.5391061132443878,
  'significant': True,
  'trait_id': 'facet_excitement_seeking'},
 {'category': 'personality',
  'name': 'Outgoing',
  'percentile': 0.3067032961897703,
  'significant': True,
  'trait_id': 'facet_friendliness'},
 {'category': 'personality',
  'name': 'Gregariousness',
  'percentile': 0.39008891803868134,
  'significant': True,
  'trait_id': 'facet_gregariousness'}]

**I want assertiveness, cheerfulness, outgoing, and gregariousness. At profile["personality"][2]["children"] 1,2,4,5.**

In [0]:
profile["personality"][3]["children"]

[{'category': 'personality',
  'name': 'Altruism',
  'percentile': 0.632121067052978,
  'significant': True,
  'trait_id': 'facet_altruism'},
 {'category': 'personality',
  'name': 'Cooperation',
  'percentile': 0.8810089590487906,
  'significant': True,
  'trait_id': 'facet_cooperation'},
 {'category': 'personality',
  'name': 'Modesty',
  'percentile': 0.6443501657833555,
  'significant': True,
  'trait_id': 'facet_modesty'},
 {'category': 'personality',
  'name': 'Uncompromising',
  'percentile': 0.9762650875871135,
  'significant': True,
  'trait_id': 'facet_morality'},
 {'category': 'personality',
  'name': 'Sympathy',
  'percentile': 0.6801664113375632,
  'significant': True,
  'trait_id': 'facet_sympathy'},
 {'category': 'personality',
  'name': 'Trust',
  'percentile': 0.6456871382279997,
  'significant': True,
  'trait_id': 'facet_trust'}]

**I want modesty, uncompromising, sympathy, trust. At profile["personality"][3]["children"] 2,3,4,5.**


In [0]:
profile["personality"][4]["children"]

[{'category': 'personality',
  'name': 'Fiery',
  'percentile': 0.003668414400169684,
  'significant': True,
  'trait_id': 'facet_anger'},
 {'category': 'personality',
  'name': 'Prone to worry',
  'percentile': 0.05804500799819429,
  'significant': True,
  'trait_id': 'facet_anxiety'},
 {'category': 'personality',
  'name': 'Melancholy',
  'percentile': 0.37817562010298633,
  'significant': True,
  'trait_id': 'facet_depression'},
 {'category': 'personality',
  'name': 'Immoderation',
  'percentile': 0.4259049231607284,
  'significant': True,
  'trait_id': 'facet_immoderation'},
 {'category': 'personality',
  'name': 'Self-consciousness',
  'percentile': 0.297625002792103,
  'significant': True,
  'trait_id': 'facet_self_consciousness'},
 {'category': 'personality',
  'name': 'Susceptible to stress',
  'percentile': 0.07255870151793248,
  'significant': True,
  'trait_id': 'facet_vulnerability'}]

**I want fiery, melancholy, immoderation, self-conscious, and stress. profile["personality"][4]["children"] 0,2,3,4,5.**

**This gives me a total of 17 personality features. There are also the two sentiment features which gives me a total of 19. I should note that there are only 12 audio features.**

In [0]:
# Testing grabbing all the traits
traits = {}
traits["artistic"] = profile["personality"][0]["children"][1]["percentile"]
traits["emotion"] = profile["personality"][0]["children"][2]["percentile"]
traits["imagination"] = profile["personality"][0]["children"][3]["percentile"]
traits["defiance"] = profile["personality"][0]["children"][5]["percentile"]

traits["assertive"] = profile["personality"][2]["children"][1]["percentile"]
traits["cheerful"] = profile["personality"][2]["children"][2]["percentile"]
traits["outgoing"] = profile["personality"][2]["children"][4]["percentile"]
traits["gregarious"] = profile["personality"][2]["children"][5]["percentile"]

traits["modesty"] = profile["personality"][3]["children"][2]["percentile"]
traits["stubborn"] = profile["personality"][3]["children"][3]["percentile"]
traits["sympathy"] = profile["personality"][3]["children"][4]["percentile"]
traits["trust"] = profile["personality"][3]["children"][5]["percentile"]

traits["fiery"] = profile["personality"][4]["children"][0]["percentile"]
traits["melancholy"] = profile["personality"][4]["children"][2]["percentile"]
traits["immoderation"] = profile["personality"][4]["children"][3]["percentile"]
traits["self-conscious"] = profile["personality"][4]["children"][4]["percentile"]
traits["stress"] = profile["personality"][4]["children"][5]["percentile"]

traits

{'artistic': 0.9877741037522365,
 'assertive': 0.48057541931441816,
 'cheerful': 0.6921443916285832,
 'defiance': 0.8182154004283968,
 'emotion': 0.607614001899865,
 'fiery': 0.003668414400169684,
 'gregarious': 0.39008891803868134,
 'imagination': 0.8574144920593829,
 'immoderation': 0.4259049231607284,
 'melancholy': 0.37817562010298633,
 'modesty': 0.6443501657833555,
 'outgoing': 0.3067032961897703,
 'self-conscious': 0.297625002792103,
 'stress': 0.07255870151793248,
 'stubborn': 0.9762650875871135,
 'sympathy': 0.6801664113375632,
 'trust': 0.6456871382279997}

In [0]:
# Grabbing the 10 chosen personality traits for each lyric
from ibm_watson import ApiException

personality = []

for lyric in test_df["lyrics"]:
  traits = {}

  try:
    profile = personality_insights.profile(
          lyric,
          accept='application/json',
          content_type='text/html',
    ).get_result()["personality"]

    traits["artistic"] = profile[0]["children"][1]["percentile"]
    traits["emotion"] = profile[0]["children"][2]["percentile"]
    traits["imagination"] = profile[0]["children"][3]["percentile"]
    traits["defiance"] = profile[0]["children"][5]["percentile"]

    traits["assertive"] = profile[2]["children"][1]["percentile"]
    traits["cheerful"] = profile[2]["children"][2]["percentile"]
    traits["outgoing"] = profile[2]["children"][4]["percentile"]
    traits["gregarious"] = profile[2]["children"][5]["percentile"]

    traits["modesty"] = profile[3]["children"][2]["percentile"]
    traits["stubborn"] = profile[3]["children"][3]["percentile"]
    traits["sympathy"] = profile[3]["children"][4]["percentile"]
    traits["trust"] = profile[3]["children"][5]["percentile"]

    traits["fiery"] = profile[4]["children"][0]["percentile"]
    traits["melancholy"] = profile[4]["children"][2]["percentile"]
    traits["immoderation"] = profile[4]["children"][3]["percentile"]
    traits["self-conscious"] = profile[4]["children"][4]["percentile"]
    traits["stress"] = profile[4]["children"][5]["percentile"]

    personality.append(traits)

  # To continue the program if an ApiException occurrs. Will append an empty 
  # dictionary. The API requires at least 100 tokens to do an analysis, which
  # not all my lyrics might contain.
  except ApiException:
    personality.append({})

  time.sleep(0.1)

ERROR:root:The number of words 9 is less than the minimum number of words required for analysis: 100
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: The number of words 9 is less than the minimum number of words required for analysis: 100, Code: 400 , X-global-transaction-id: d9f868eacf4c099952c6bc3e423a489c
ERROR:root:The number of words 83 is less than the minimum number of words required for analysis: 100
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: The number of words 83 is less than the minimum number of words required for analysis: 100, Code: 400 , X-global-transaction

In [0]:
personality_df = pd.json_normalize(personality)
personality_df

,artistic,emotion,imagination,defiance,assertive,cheerful,outgoing,gregarious,modesty,stubborn,sympathy,trust,fiery,melancholy,immoderation,self-conscious,stress
0,0.987774,0.607614,0.857414,0.818215,0.480575,0.692144,0.306703,0.390089,0.644350,0.976265,0.680166,0.645687,0.003668,0.378176,0.425905,0.297625,0.072559
1,0.710470,0.825304,0.210163,0.232451,0.192279,0.842394,0.352414,0.409357,0.953747,0.954188,0.377419,0.165527,0.071750,0.406895,0.296371,0.375357,0.346988
2,0.891084,0.979085,0.727160,0.188132,0.075599,0.750602,0.084310,0.089439,0.997576,0.948891,0.881508,0.038746,0.276015,0.995001,0.576368,0.885831,0.976094
3,0.978994,0.186366,0.845272,0.622505,0.210006,0.879997,0.664401,0.751250,0.561308,0.800438,0.161357,0.560406,0.033857,0.412794,0.906588,0.346867,0.117505
4,0.999803,0.964237,0.689671,0.988952,0.000377,0.435559,0.002287,0.000682,0.997326,0.997298,0.894385,0.242191,0.005962,0.980147,0.527485,0.995966,0.971579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,0.999076,0.907450,0.607554,0.919184,0.073696,0.320600,0.047448,0.012190,0.953025,0.970846,0.782390,0.092330,0.065991,0.894144,0.248405,0.883670,0.621691
505,0.996684,0.037910,0.983827,0.945210,0.079179,0.374966,0.045602,0.069722,0.105508,0.087249,0.158819,0.010982,0.517425,0.949973,0.757508,0.953037,0.940817
506,0.985637,0.956805,0.994814,0.877157,0.013549,0.391997,0.048806,0.066037,0.797305,0.249545,0.863495,0.019229,0.863935,0.999886,0.925857,0.979232,0.999283
507,0.107095,0.197436,0.022112,0.180456,0.518896,0.880641,0.883176,0.976546,0.994459,0.942164,0.072255,0.123573,0.116184,0.414304,0.799621,0.060368,0.189484


In [0]:
test_df = test_df.merge(personality_df, left_index=True, right_index=True,
                        how="outer")
test_df

,id,artist,song,album,date,explicit,popularity,key,mode,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,chain_lyrics,lyrics,polarity,magnitude,artistic,emotion,imagination,defiance,assertive,cheerful,outgoing,gregarious,modesty,stubborn,sympathy,trust,fiery,melancholy,immoderation,self-conscious,stress
0,1yTTMcUhL7rtz08Dsgb7Qb,Maren Morris,The Bones,The Bones (with Hozier),2019-10-04,False,77,11,0,4,0.286000,0.561,0.597,0.000000,0.0979,-6.000,0.0405,76.826,0.355,were in the homestretch of the high times \n w...,were in the homestretch of the high times we t...,-1.0,1.0,0.987774,0.607614,0.857414,0.818215,0.480575,0.692144,0.306703,0.390089,0.644350,0.976265,0.680166,0.645687,0.003668,0.378176,0.425905,0.297625,0.072559
1,6UnCGAEmrbGIOSmGRZQ1M2,Maggie Rogers,Light On,Heard It In A Past Life,2019-01-18,False,74,2,1,4,0.201000,0.657,0.569,0.000014,0.1260,-6.287,0.0542,102.054,0.399,would you believe me now \n if i told you i go...,would you believe me now if i told you i got c...,-1.0,0.1,0.710470,0.825304,0.210163,0.232451,0.192279,0.842394,0.352414,0.409357,0.953747,0.954188,0.377419,0.165527,0.071750,0.406895,0.296371,0.375357,0.346988
2,3V0nnQhqvbE3JmiDdnzQFQ,Noah Cyrus,July,July (feat. Leon Bridges),2019-11-08,False,73,9,0,4,0.853000,0.708,0.211,0.000000,0.1310,-9.603,0.0424,72.500,0.362,ive been holding my breath \n ive been countin...,ive been holding my breath ive been counting t...,1.0,0.1,0.891084,0.979085,0.727160,0.188132,0.075599,0.750602,0.084310,0.089439,0.997576,0.948891,0.881508,0.038746,0.276015,0.995001,0.576368,0.885831,0.976094
3,6ET9kf9riLETWs9lePUEAI,Kacey Musgraves,Slow Burn,Golden Hour,2018-03-30,False,69,2,1,4,0.462000,0.555,0.430,0.000191,0.0869,-9.105,0.0295,147.613,0.274,born in a hurry always late \n havent been ear...,born in a hurry always late havent been early ...,-1.0,0.2,0.978994,0.186366,0.845272,0.622505,0.210006,0.879997,0.664401,0.751250,0.561308,0.800438,0.161357,0.560406,0.033857,0.412794,0.906588,0.346867,0.117505
4,1HNywa8Bk0TmzRwQrzhtPo,John Mayer,I Guess I Just Feel Like,I Guess I Just Feel Like,2019-02-22,False,73,4,1,4,0.615000,0.556,0.528,0.096400,0.1220,-8.033,0.0266,159.821,0.163,i guess i just feel like \n i guess i just fee...,i guess i just feel like i guess i just feel l...,1.0,0.4,0.999803,0.964237,0.689671,0.988952,0.000377,0.435559,0.002287,0.000682,0.997326,0.997298,0.894385,0.242191,0.005962,0.980147,0.527485,0.995966,0.971579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,7x8dCjCr0x6x2lXKujYD34,Foo Fighters,The Pretender,"Echoes, Silence, Patience & Grace",2007-09-25,False,75,9,1,4,0.000917,0.433,0.959,0.000000,0.0280,-4.040,0.0431,172.984,0.365,keep you in the dark \n you know they all pret...,keep you in the dark you know they all pretend...,-1.0,0.9,0.999076,0.907450,0.607554,0.919184,0.073696,0.320600,0.047448,0.012190,0.953025,0.970846,0.782390,0.092330,0.065991,0.894144,0.248405,0.883670,0.621691
505,3eDJXbkSGulxTuB2L7fP9B,Crobot,Low Life,Motherbrain,2019-08-23,False,59,6,1,4,0.000800,0.546,0.911,0.000077,0.1310,-4.801,0.0620,76.008,0.757,turn off your television \n theyre selling sup...,turn off your television theyre selling supers...,-1.0,1.1,0.996684,0.037910,0.983827,0.945210,0.079179,0.374966,0.045602,0.069722,0.105508,0.087249,0.158819,0.010982,0.517425,0.949973,0.757508,0.953037,0.940817
506,5eek2X5459T1HoYJk2CKXv,Papa Roach,Last Resort,Infest,2000-04-25,True,17,4,0,4,0.000504,0.581,0.887,0.001110,0.2680,-3.659,0.0624,90.578,0.724,cut my life into pieces \n this is my last res...,cut my life into pieces this is my last resort...,-1.0,1.6,0.985637,0.956805,0.994814,0.877157,0.013549,0.391997,0.048806,0.066037,0.797305,0.249545,0.863495,0.019229,0.863935,0.999886,0.925857,0.979232,0.999283
507,24tO365YW6lcZr1hN4Ukzj,Silverchair,Tomorrow,Frogstomp (Deluxe Edition) [Remastered],199

In [0]:
# So I lost 8 songs that weren't 100 words or longer leaving me with 500 total
test_df[test_df["fiery"].notnull()]

,id,artist,song,album,date,explicit,popularity,key,mode,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,chain_lyrics,lyrics,polarity,magnitude,artistic,emotion,imagination,defiance,assertive,cheerful,outgoing,gregarious,modesty,stubborn,sympathy,trust,fiery,melancholy,immoderation,self-conscious,stress
0,1yTTMcUhL7rtz08Dsgb7Qb,Maren Morris,The Bones,The Bones (with Hozier),2019-10-04,False,77,11,0,4,0.286000,0.561,0.597,0.000000,0.0979,-6.000,0.0405,76.826,0.355,were in the homestretch of the high times \n w...,were in the homestretch of the high times we t...,-1.0,1.0,0.987774,0.607614,0.857414,0.818215,0.480575,0.692144,0.306703,0.390089,0.644350,0.976265,0.680166,0.645687,0.003668,0.378176,0.425905,0.297625,0.072559
1,6UnCGAEmrbGIOSmGRZQ1M2,Maggie Rogers,Light On,Heard It In A Past Life,2019-01-18,False,74,2,1,4,0.201000,0.657,0.569,0.000014,0.1260,-6.287,0.0542,102.054,0.399,would you believe me now \n if i told you i go...,would you believe me now if i told you i got c...,-1.0,0.1,0.710470,0.825304,0.210163,0.232451,0.192279,0.842394,0.352414,0.409357,0.953747,0.954188,0.377419,0.165527,0.071750,0.406895,0.296371,0.375357,0.346988
2,3V0nnQhqvbE3JmiDdnzQFQ,Noah Cyrus,July,July (feat. Leon Bridges),2019-11-08,False,73,9,0,4,0.853000,0.708,0.211,0.000000,0.1310,-9.603,0.0424,72.500,0.362,ive been holding my breath \n ive been countin...,ive been holding my breath ive been counting t...,1.0,0.1,0.891084,0.979085,0.727160,0.188132,0.075599,0.750602,0.084310,0.089439,0.997576,0.948891,0.881508,0.038746,0.276015,0.995001,0.576368,0.885831,0.976094
3,6ET9kf9riLETWs9lePUEAI,Kacey Musgraves,Slow Burn,Golden Hour,2018-03-30,False,69,2,1,4,0.462000,0.555,0.430,0.000191,0.0869,-9.105,0.0295,147.613,0.274,born in a hurry always late \n havent been ear...,born in a hurry always late havent been early ...,-1.0,0.2,0.978994,0.186366,0.845272,0.622505,0.210006,0.879997,0.664401,0.751250,0.561308,0.800438,0.161357,0.560406,0.033857,0.412794,0.906588,0.346867,0.117505
4,1HNywa8Bk0TmzRwQrzhtPo,John Mayer,I Guess I Just Feel Like,I Guess I Just Feel Like,2019-02-22,False,73,4,1,4,0.615000,0.556,0.528,0.096400,0.1220,-8.033,0.0266,159.821,0.163,i guess i just feel like \n i guess i just fee...,i guess i just feel like i guess i just feel l...,1.0,0.4,0.999803,0.964237,0.689671,0.988952,0.000377,0.435559,0.002287,0.000682,0.997326,0.997298,0.894385,0.242191,0.005962,0.980147,0.527485,0.995966,0.971579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,7x8dCjCr0x6x2lXKujYD34,Foo Fighters,The Pretender,"Echoes, Silence, Patience & Grace",2007-09-25,False,75,9,1,4,0.000917,0.433,0.959,0.000000,0.0280,-4.040,0.0431,172.984,0.365,keep you in the dark \n you know they all pret...,keep you in the dark you know they all pretend...,-1.0,0.9,0.999076,0.907450,0.607554,0.919184,0.073696,0.320600,0.047448,0.012190,0.953025,0.970846,0.782390,0.092330,0.065991,0.894144,0.248405,0.883670,0.621691
505,3eDJXbkSGulxTuB2L7fP9B,Crobot,Low Life,Motherbrain,2019-08-23,False,59,6,1,4,0.000800,0.546,0.911,0.000077,0.1310,-4.801,0.0620,76.008,0.757,turn off your television \n theyre selling sup...,turn off your television theyre selling supers...,-1.0,1.1,0.996684,0.037910,0.983827,0.945210,0.079179,0.374966,0.045602,0.069722,0.105508,0.087249,0.158819,0.010982,0.517425,0.949973,0.757508,0.953037,0.940817
506,5eek2X5459T1HoYJk2CKXv,Papa Roach,Last Resort,Infest,2000-04-25,True,17,4,0,4,0.000504,0.581,0.887,0.001110,0.2680,-3.659,0.0624,90.578,0.724,cut my life into pieces \n this is my last res...,cut my life into pieces this is my last resort...,-1.0,1.6,0.985637,0.956805,0.994814,0.877157,0.013549,0.391997,0.048806,0.066037,0.797305,0.249545,0.863495,0.019229,0.863935,0.999886,0.925857,0.979232,0.999283
507,24tO365YW6lcZr1hN4Ukzj,Silverchair,Tomorrow,Frogstomp (Deluxe Edition) [Remastered],199

In [0]:
# This is my final test dataset, complete.
test_df = test_df[test_df["fiery"].notnull()]
test_df = test_df.reset_index().iloc[:,1:]
test_df

,id,artist,song,album,date,explicit,popularity,key,mode,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,chain_lyrics,lyrics,polarity,magnitude,artistic,emotion,imagination,defiance,assertive,cheerful,outgoing,gregarious,modesty,stubborn,sympathy,trust,fiery,melancholy,immoderation,self-conscious,stress
0,1yTTMcUhL7rtz08Dsgb7Qb,Maren Morris,The Bones,The Bones (with Hozier),2019-10-04,False,77,11,0,4,0.286000,0.561,0.597,0.000000,0.0979,-6.000,0.0405,76.826,0.355,were in the homestretch of the high times \n w...,were in the homestretch of the high times we t...,-1.0,1.0,0.987774,0.607614,0.857414,0.818215,0.480575,0.692144,0.306703,0.390089,0.644350,0.976265,0.680166,0.645687,0.003668,0.378176,0.425905,0.297625,0.072559
1,6UnCGAEmrbGIOSmGRZQ1M2,Maggie Rogers,Light On,Heard It In A Past Life,2019-01-18,False,74,2,1,4,0.201000,0.657,0.569,0.000014,0.1260,-6.287,0.0542,102.054,0.399,would you believe me now \n if i told you i go...,would you believe me now if i told you i got c...,-1.0,0.1,0.710470,0.825304,0.210163,0.232451,0.192279,0.842394,0.352414,0.409357,0.953747,0.954188,0.377419,0.165527,0.071750,0.406895,0.296371,0.375357,0.346988
2,3V0nnQhqvbE3JmiDdnzQFQ,Noah Cyrus,July,July (feat. Leon Bridges),2019-11-08,False,73,9,0,4,0.853000,0.708,0.211,0.000000,0.1310,-9.603,0.0424,72.500,0.362,ive been holding my breath \n ive been countin...,ive been holding my breath ive been counting t...,1.0,0.1,0.891084,0.979085,0.727160,0.188132,0.075599,0.750602,0.084310,0.089439,0.997576,0.948891,0.881508,0.038746,0.276015,0.995001,0.576368,0.885831,0.976094
3,6ET9kf9riLETWs9lePUEAI,Kacey Musgraves,Slow Burn,Golden Hour,2018-03-30,False,69,2,1,4,0.462000,0.555,0.430,0.000191,0.0869,-9.105,0.0295,147.613,0.274,born in a hurry always late \n havent been ear...,born in a hurry always late havent been early ...,-1.0,0.2,0.978994,0.186366,0.845272,0.622505,0.210006,0.879997,0.664401,0.751250,0.561308,0.800438,0.161357,0.560406,0.033857,0.412794,0.906588,0.346867,0.117505
4,1HNywa8Bk0TmzRwQrzhtPo,John Mayer,I Guess I Just Feel Like,I Guess I Just Feel Like,2019-02-22,False,73,4,1,4,0.615000,0.556,0.528,0.096400,0.1220,-8.033,0.0266,159.821,0.163,i guess i just feel like \n i guess i just fee...,i guess i just feel like i guess i just feel l...,1.0,0.4,0.999803,0.964237,0.689671,0.988952,0.000377,0.435559,0.002287,0.000682,0.997326,0.997298,0.894385,0.242191,0.005962,0.980147,0.527485,0.995966,0.971579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,7x8dCjCr0x6x2lXKujYD34,Foo Fighters,The Pretender,"Echoes, Silence, Patience & Grace",2007-09-25,False,75,9,1,4,0.000917,0.433,0.959,0.000000,0.0280,-4.040,0.0431,172.984,0.365,keep you in the dark \n you know they all pret...,keep you in the dark you know they all pretend...,-1.0,0.9,0.999076,0.907450,0.607554,0.919184,0.073696,0.320600,0.047448,0.012190,0.953025,0.970846,0.782390,0.092330,0.065991,0.894144,0.248405,0.883670,0.621691
496,3eDJXbkSGulxTuB2L7fP9B,Crobot,Low Life,Motherbrain,2019-08-23,False,59,6,1,4,0.000800,0.546,0.911,0.000077,0.1310,-4.801,0.0620,76.008,0.757,turn off your television \n theyre selling sup...,turn off your television theyre selling supers...,-1.0,1.1,0.996684,0.037910,0.983827,0.945210,0.079179,0.374966,0.045602,0.069722,0.105508,0.087249,0.158819,0.010982,0.517425,0.949973,0.757508,0.953037,0.940817
497,5eek2X5459T1HoYJk2CKXv,Papa Roach,Last Resort,Infest,2000-04-25,True,17,4,0,4,0.000504,0.581,0.887,0.001110,0.2680,-3.659,0.0624,90.578,0.724,cut my life into pieces \n this is my last res...,cut my life into pieces this is my last resort...,-1.0,1.6,0.985637,0.956805,0.994814,0.877157,0.013549,0.391997,0.048806,0.066037,0.797305,0.249545,0.863495,0.019229,0.863935,0.999886,0.925857,0.979232,0.999283
498,24tO365YW6lcZr1hN4Ukzj,Silverchair,Tomorrow,Frogstomp (Deluxe Edition) [Remastered],199

In [0]:
test_df.to_csv('/content/drive/My Drive/spotify_test_data.csv', index=False)

# Final Cleaning

In [0]:
test_df = pd.read_csv('/content/drive/My Drive/spotify_test_data.csv')
test_df.head()

,id,artist,song,album,date,explicit,popularity,key,mode,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,chain_lyrics,lyrics,polarity,magnitude,artistic,emotion,imagination,defiance,assertive,cheerful,outgoing,gregarious,modesty,stubborn,sympathy,trust,fiery,melancholy,immoderation,self-conscious,stress
0,1yTTMcUhL7rtz08Dsgb7Qb,Maren Morris,The Bones,The Bones (with Hozier),2019-10-04,False,77,11,0,4,0.286,0.561,0.597,0.000000,0.0979,-6.000,0.0405,76.826,0.355,were in the homestretch of the high times \n w...,were in the homestretch of the high times we t...,-1.0,1.0,0.987774,0.607614,0.857414,0.818215,0.480575,0.692144,0.306703,0.390089,0.644350,0.976265,0.680166,0.645687,0.003668,0.378176,0.425905,0.297625,0.072559
1,6UnCGAEmrbGIOSmGRZQ1M2,Maggie Rogers,Light On,Heard It In A Past Life,2019-01-18,False,74,2,1,4,0.201,0.657,0.569,0.000014,0.1260,-6.287,0.0542,102.054,0.399,would you believe me now \n if i told you i go...,would you believe me now if i told you i got c...,-1.0,0.1,0.710470,0.825304,0.210163,0.232451,0.192279,0.842394,0.352414,0.409357,0.953747,0.954188,0.377419,0.165527,0.071750,0.406895,0.296371,0.375357,0.346988
2,3V0nnQhqvbE3JmiDdnzQFQ,Noah Cyrus,July,July (feat. Leon Bridges),2019-11-08,False,73,9,0,4,0.853,0.708,0.211,0.000000,0.1310,-9.603,0.0424,72.500,0.362,ive been holding my breath \n ive been countin...,ive been holding my breath ive been counting t...,1.0,0.1,0.891084,0.979085,0.727160,0.188132,0.075599,0.750602,0.084310,0.089439,0.997576,0.948891,0.881508,0.038746,0.276015,0.995001,0.576368,0.885831,0.976094
3,6ET9kf9riLETWs9lePUEAI,Kacey Musgraves,Slow Burn,Golden Hour,2018-03-30,False,69,2,1,4,0.462,0.555,0.430,0.000191,0.0869,-9.105,0.0295,147.613,0.274,born in a hurry always late \n havent been ear...,born in a hurry always late havent been early ...,-1.0,0.2,0.978994,0.186366,0.845272,0.622505,0.210006,0.879997,0.664401,0.751250,0.561308,0.800438,0.161357,0.560406,0.033857,0.412794,0.906588,0.346867,0.117505
4,1HNywa8Bk0TmzRwQrzhtPo,John Mayer,I Guess I Just Feel Like,I Guess I Just Feel Like,2019-02-22,False,73,4,1,4,0.615,0.556,0.528,0.096400,0.1220,-8.033,0.0266,159.821,0.163,i guess i just feel like \n i guess i just fee...,i guess i just feel like i guess i just feel l...,1.0,0.4,0.999803,0.964237,0.689671,0.988952,0.000377,0.435559,0.002287,0.000682,0.997326,0.997298,0.894385,0.242191,0.005962,0.980147,0.527485,0.995966,0.971579


In [0]:
test_df.isnull().sum()

id                  0
artist              0
song                0
album               0
date                0
explicit            0
popularity          0
key                 0
mode                0
time_signature      0
acousticness        0
danceability        0
energy              0
instrumentalness    0
liveness            0
loudness            0
speechiness         0
tempo               0
valence             0
chain_lyrics        0
lyrics              0
polarity            0
magnitude           0
artistic            0
emotion             0
imagination         0
defiance            0
assertive           0
cheerful            0
outgoing            0
gregarious          0
modesty             0
stubborn            0
sympathy            0
trust               0
fiery               0
melancholy          0
immoderation        0
self-conscious      0
stress              0
dtype: int64

In [0]:
# I want the explicit column to be zeros and ones
test_df["explicit"] = test_df["explicit"].astype("int64")
test_df.head()

,id,artist,song,album,date,explicit,popularity,key,mode,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,chain_lyrics,lyrics,polarity,magnitude,artistic,emotion,imagination,defiance,assertive,cheerful,outgoing,gregarious,modesty,stubborn,sympathy,trust,fiery,melancholy,immoderation,self-conscious,stress
0,1yTTMcUhL7rtz08Dsgb7Qb,Maren Morris,The Bones,The Bones (with Hozier),2019-10-04,0,77,11,0,4,0.286,0.561,0.597,0.000000,0.0979,-6.000,0.0405,76.826,0.355,were in the homestretch of the high times \n w...,were in the homestretch of the high times we t...,-1.0,1.0,0.987774,0.607614,0.857414,0.818215,0.480575,0.692144,0.306703,0.390089,0.644350,0.976265,0.680166,0.645687,0.003668,0.378176,0.425905,0.297625,0.072559
1,6UnCGAEmrbGIOSmGRZQ1M2,Maggie Rogers,Light On,Heard It In A Past Life,2019-01-18,0,74,2,1,4,0.201,0.657,0.569,0.000014,0.1260,-6.287,0.0542,102.054,0.399,would you believe me now \n if i told you i go...,would you believe me now if i told you i got c...,-1.0,0.1,0.710470,0.825304,0.210163,0.232451,0.192279,0.842394,0.352414,0.409357,0.953747,0.954188,0.377419,0.165527,0.071750,0.406895,0.296371,0.375357,0.346988
2,3V0nnQhqvbE3JmiDdnzQFQ,Noah Cyrus,July,July (feat. Leon Bridges),2019-11-08,0,73,9,0,4,0.853,0.708,0.211,0.000000,0.1310,-9.603,0.0424,72.500,0.362,ive been holding my breath \n ive been countin...,ive been holding my breath ive been counting t...,1.0,0.1,0.891084,0.979085,0.727160,0.188132,0.075599,0.750602,0.084310,0.089439,0.997576,0.948891,0.881508,0.038746,0.276015,0.995001,0.576368,0.885831,0.976094
3,6ET9kf9riLETWs9lePUEAI,Kacey Musgraves,Slow Burn,Golden Hour,2018-03-30,0,69,2,1,4,0.462,0.555,0.430,0.000191,0.0869,-9.105,0.0295,147.613,0.274,born in a hurry always late \n havent been ear...,born in a hurry always late havent been early ...,-1.0,0.2,0.978994,0.186366,0.845272,0.622505,0.210006,0.879997,0.664401,0.751250,0.561308,0.800438,0.161357,0.560406,0.033857,0.412794,0.906588,0.346867,0.117505
4,1HNywa8Bk0TmzRwQrzhtPo,John Mayer,I Guess I Just Feel Like,I Guess I Just Feel Like,2019-02-22,0,73,4,1,4,0.615,0.556,0.528,0.096400,0.1220,-8.033,0.0266,159.821,0.163,i guess i just feel like \n i guess i just fee...,i guess i just feel like i guess i just feel l...,1.0,0.4,0.999803,0.964237,0.689671,0.988952,0.000377,0.435559,0.002287,0.000682,0.997326,0.997298,0.894385,0.242191,0.005962,0.980147,0.527485,0.995966,0.971579


In [0]:
test_df["date"]

0      2019-10-04
1      2019-01-18
2      2019-11-08
3      2018-03-30
4      2019-02-22
          ...    
495    2007-09-25
496    2019-08-23
497    2000-04-25
498    1995-03-27
499    2001-01-01
Name: date, Length: 500, dtype: object

In [0]:
# I want my date to be a datetime object
test_df["date"] = pd.to_datetime(test_df["date"])
test_df["date"]

0     2019-10-04
1     2019-01-18
2     2019-11-08
3     2018-03-30
4     2019-02-22
         ...    
495   2007-09-25
496   2019-08-23
497   2000-04-25
498   1995-03-27
499   2001-01-01
Name: date, Length: 500, dtype: datetime64[ns]

**I do not want to use artist, song, or album as features, because there will undoubtedly be values not present in both training and test, and in any potential future data to predict on. However, I will keep them in the dataframe. And now that I am done, I no longer need to unique spotify id**

In [0]:
test_df  = test_df.iloc[:,1:]
test_df

,artist,song,album,date,explicit,popularity,key,mode,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,chain_lyrics,lyrics,polarity,magnitude,artistic,emotion,imagination,defiance,assertive,cheerful,outgoing,gregarious,modesty,stubborn,sympathy,trust,fiery,melancholy,immoderation,self-conscious,stress
0,Maren Morris,The Bones,The Bones (with Hozier),2019-10-04,0,77,11,0,4,0.286000,0.561,0.597,0.000000,0.0979,-6.000,0.0405,76.826,0.355,were in the homestretch of the high times \n w...,were in the homestretch of the high times we t...,-1.0,1.0,0.987774,0.607614,0.857414,0.818215,0.480575,0.692144,0.306703,0.390089,0.644350,0.976265,0.680166,0.645687,0.003668,0.378176,0.425905,0.297625,0.072559
1,Maggie Rogers,Light On,Heard It In A Past Life,2019-01-18,0,74,2,1,4,0.201000,0.657,0.569,0.000014,0.1260,-6.287,0.0542,102.054,0.399,would you believe me now \n if i told you i go...,would you believe me now if i told you i got c...,-1.0,0.1,0.710470,0.825304,0.210163,0.232451,0.192279,0.842394,0.352414,0.409357,0.953747,0.954188,0.377419,0.165527,0.071750,0.406895,0.296371,0.375357,0.346988
2,Noah Cyrus,July,July (feat. Leon Bridges),2019-11-08,0,73,9,0,4,0.853000,0.708,0.211,0.000000,0.1310,-9.603,0.0424,72.500,0.362,ive been holding my breath \n ive been countin...,ive been holding my breath ive been counting t...,1.0,0.1,0.891084,0.979085,0.727160,0.188132,0.075599,0.750602,0.084310,0.089439,0.997576,0.948891,0.881508,0.038746,0.276015,0.995001,0.576368,0.885831,0.976094
3,Kacey Musgraves,Slow Burn,Golden Hour,2018-03-30,0,69,2,1,4,0.462000,0.555,0.430,0.000191,0.0869,-9.105,0.0295,147.613,0.274,born in a hurry always late \n havent been ear...,born in a hurry always late havent been early ...,-1.0,0.2,0.978994,0.186366,0.845272,0.622505,0.210006,0.879997,0.664401,0.751250,0.561308,0.800438,0.161357,0.560406,0.033857,0.412794,0.906588,0.346867,0.117505
4,John Mayer,I Guess I Just Feel Like,I Guess I Just Feel Like,2019-02-22,0,73,4,1,4,0.615000,0.556,0.528,0.096400,0.1220,-8.033,0.0266,159.821,0.163,i guess i just feel like \n i guess i just fee...,i guess i just feel like i guess i just feel l...,1.0,0.4,0.999803,0.964237,0.689671,0.988952,0.000377,0.435559,0.002287,0.000682,0.997326,0.997298,0.894385,0.242191,0.005962,0.980147,0.527485,0.995966,0.971579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Foo Fighters,The Pretender,"Echoes, Silence, Patience & Grace",2007-09-25,0,75,9,1,4,0.000917,0.433,0.959,0.000000,0.0280,-4.040,0.0431,172.984,0.365,keep you in the dark \n you know they all pret...,keep you in the dark you know they all pretend...,-1.0,0.9,0.999076,0.907450,0.607554,0.919184,0.073696,0.320600,0.047448,0.012190,0.953025,0.970846,0.782390,0.092330,0.065991,0.894144,0.248405,0.883670,0.621691
496,Crobot,Low Life,Motherbrain,2019-08-23,0,59,6,1,4,0.000800,0.546,0.911,0.000077,0.1310,-4.801,0.0620,76.008,0.757,turn off your television \n theyre selling sup...,turn off your television theyre selling supers...,-1.0,1.1,0.996684,0.037910,0.983827,0.945210,0.079179,0.374966,0.045602,0.069722,0.105508,0.087249,0.158819,0.010982,0.517425,0.949973,0.757508,0.953037,0.940817
497,Papa Roach,Last Resort,Infest,2000-04-25,1,17,4,0,4,0.000504,0.581,0.887,0.001110,0.2680,-3.659,0.0624,90.578,0.724,cut my life into pieces \n this is my last res...,cut my life into pieces this is my last resort...,-1.0,1.6,0.985637,0.956805,0.994814,0.877157,0.013549,0.391997,0.048806,0.066037,0.797305,0.249545,0.863495,0.019229,0.863935,0.999886,0.925857,0.979232,0.999283
498,Silverchair,Tomorrow,Frogstomp (Deluxe Edition) [Remastered],1995-03-27,0,61,2,1,3,0.000658,0.292,0.714,0.000109,0.1020,-4.919,0.0316,151.412,0.462,its twelve oclock and its a wonderful day \n i...,its twelve oclock and its a wonderful day i kn...,-1.0,0.5,0.107095,0.197436,0.022112,0.180456,0.518896,0.88064

In [0]:
# Finally I am done
test_df.to_csv('/content/drive/My Drive/spotify_test_data.csv', index=False)